In [1]:
import opensim as osim
from connections import AWS
import xml.etree.ElementTree as ET
from biomech.processing.trc import *

In [2]:
""" INITIALIZE AWS CONNECTION """
aws_connection = AWS()
aws_connection.connect()

[AWS]: Port 5433 is in use by process python3.11 (PID 7866). Killing it.
[AWS]: Connected to RDS endpoint.


$\textbf{OpenSim: Scaling (Development)}$

In [3]:
## OUTLINE

# connect to AWS [DONE]
# load all subject info [DONE]
# iterate through subjects: 
    # load default model, scaling template for throwing hand --> write to local dir?
    # load static trial for subject --> write to local dir?
    # run opensim scaling tool 

# NOTE: all files (scaled model and static marker locations) to be written to subject-specific `osim` directory
# NOTE: scaling is uniformly applied to all axes based on marker pairs

In [4]:
# load all subject info
subject_info = aws_connection.load_subject_info()
MASS_PERC = 0.0492

# set XML dir path 
XML_DIR = 'biomechanics/xml_templates'
OSIM_DIR = 'biomechanics/opensim'            # this applies to local and S3 bucket

In [7]:
# initialize error log
error_log = []

# iterate through subjects
for subject in subject_info['subject_id'].unique():
    
    try:
        # get subject info (ID -- string; throwing hand)
        subject_id = str(subject)
        throwing_hand = subject_info[subject_info['subject_id'] == subject]['throws'].values[0]

        # load scaling template, default model, & static trial
        scaling_template = aws_connection.load_xml_from_s3(f'{XML_DIR}/scaling_{throwing_hand}.xml')
        default_model = aws_connection.load_xml_from_s3(f'{OSIM_DIR}/models_arm_{throwing_hand}.osim')
        static_trial = aws_connection.load_s3_object(f'biomechanics/subjects/{subject_id}/trc_raw/{subject_id}_static.trc', return_info=False)

        # update scaling template w/ subject info
        scaling_template.find(".//mass").text = str(subject_info[subject_info['subject_id'] == subject]['mass'].values[0] * MASS_PERC)
        scaling_template.find(".//height").text = str(subject_info[subject_info['subject_id'] == subject]['height'].values[0])
        scaling_template.find(".//age").text = str(subject_info[subject_info['subject_id'] == subject]['age'].values[0])

        """ Write XML to local `opensim` dir (enables use of osim API tools) """
        # scaling template
        scaling_template_tree = ET.ElementTree(scaling_template)
        scaling_template_tree.write(f'scaling_{throwing_hand}.xml')

        # default model
        default_model_tree = ET.ElementTree(default_model)
        default_model_tree.write(f'models_arm_{throwing_hand}.osim')

        # static trial
        static_trial_path = 'static_trial.trc'
        static_trial_body = parse_trc_body(static_trial, throwing_hand=throwing_hand, filter_markers=False)
        write_to_trc(static_trial_path, static_trial_body, throwing_hand=throwing_hand, filter_markers=False)

        """ Run OpenSim scaling tool """
        print(f"[info] Subject ID: {subject_id}")                               # use this to help log errors by subject
        scaling_tool = osim.ScaleTool(f'scaling_{throwing_hand}.xml')
        scaling_tool.run()

        """ Write each file to S3 bucket (scaled model, static markers, and scaling template) """
        # set upload directory
        UPLOAD_DIR = f'biomechanics/subjects/{subject_id}/osim'
        # TEST_DIR = 'test'

        # scaled model
        with open(f'models_arm_{throwing_hand}_scaled.osim', 'r') as f:
            scaled_model_content = f.read()
        aws_connection.upload_to_s3(
            scaled_model_content,
            f'{UPLOAD_DIR}/{subject_id}_scaled_model.osim', 
        )
        
        # static markers
        with open('static_markers.sto', 'r') as f:
            static_markers_content = f.read()
        aws_connection.upload_to_s3(
            static_markers_content,
            f'{UPLOAD_DIR}/{subject_id}_static_markers.sto', 
        )
        
        # scaling template
        with open(f'scaling_{throwing_hand}.xml', 'r') as f:
            scaling_template_content = f.read()
        aws_connection.upload_to_s3(
            scaling_template_content,
            f'{UPLOAD_DIR}/{subject_id}_scaling_template.xml', 
        )

    except Exception as e:
        print(f"Error processing subject {subject_id}: {e}")

        # log error
        error_log.append({
            'subject_id': subject_id,
            'error': str(e)
        })

        continue

[info] Subject ID: 2609
[info] Processing subject models_arm_right_scaled...
[info] Step 1: Loading generic model
[info] Loaded model rhp_arm from file /Users/cmoore/Library/Mobile Documents/com~apple~CloudDocs/Human RITHM/GitHub/pitch-ml/prod/biomech/scaling/models_arm_right.osim
[warning] Couldn't find file 'humerus_rv.vtp'.
[warning] Couldn't find file 'ulna_rv.vtp'.
[warning] Couldn't find file 'radius_rv.vtp'.
[warning] Couldn't find file 'pisiform_rvs.vtp'.
[warning] Couldn't find file 'lunate_rvs.vtp'.
[warning] Couldn't find file 'scaphoid_rvs.vtp'.
[warning] Couldn't find file 'triquetrum_rvs.vtp'.
[warning] Couldn't find file 'hamate_rvs.vtp'.
[warning] Couldn't find file 'capitate_rvs.vtp'.
[warning] Couldn't find file 'trapezoid_rvs.vtp'.
[warning] Couldn't find file 'trapezium_rvs.vtp'.
[warning] Couldn't find file 'metacarpal2_rvs.vtp'.
[warning] Couldn't find file 'index_proximal_rvs.vtp'.
[warning] Couldn't find file 'index_medial_rvs.vtp'.
[warning] Couldn't find file 

In [9]:
# concatenate error log
error_log_df = pd.DataFrame(error_log)

# write error log to S3 bucket
aws_connection.upload_to_s3(
    error_log_df.to_csv(index=False),
    'biomechanics/subjects/summary/error_scaling_log.csv'
)

[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/summary/error_scaling_log.csv


$\textbf{Parse Scaling Errors}$

In [10]:
import re

In [11]:
def parse_scale_output(error_file: str) -> list:
    results = []
    
    # specify regex patterns
    subject_pattern = re.compile(r"Subject ID: (\d+)")
    error_pattern = re.compile(r"Frame at \(t = .*?\):.*?marker error: RMS = ([\d.]+), max = ([\d.]+) \((.*?)\)")

    # open file & read lines
    with open(error_file, 'r') as file:
        lines = file.readlines()

    # iterate through lines
    current_subject = None
    for line in lines:
        # match subject ID
        subject_match = subject_pattern.search(line)
        if subject_match:
            current_subject = subject_match.group(1)

        # match errors to patterns
        error_match = error_pattern.search(line)
        if error_match and current_subject:
            rms_error = float(error_match.group(1))
            max_error = float(error_match.group(2))
            max_error_marker = error_match.group(3)
            results.append({
                'subject_id': current_subject,
                'rms_error': rms_error,
                'max_error': max_error,
                'max_error_marker': max_error_marker
            })
            current_subject = None

    return results

In [13]:
# parse scaling results
parsed_errors = parse_scale_output(error_file='errors.txt')

# create & save dataframe to (local) csv
scaling_results = pd.DataFrame(parsed_errors).sort_values(by='rms_error', ascending=False)
scaling_results.to_csv('errors_parsed.csv', index=False)

# upload parsed errors to S3 bucket
aws_connection.upload_to_s3(
    scaling_results.to_csv(index=False),
    'biomechanics/subjects/summary/results_scaling.csv'
)

[AWS]: Uploaded object to s3://pitch-ml/biomechanics/subjects/summary/results_scaling.csv


$\textbf{Close AWS Connection}$

In [19]:
aws_connection.close()

[AWS]: Database connection closed.
[AWS]: SSH tunnel stopped.
